# Computação Gráfica - Phong lighting model

<b> Detalhes de implementação</b>
<p>Para a realização deste trablaho, foi implementado o phong lighting model com os 3 shadings: Gouraud, Phong e Flat. Essa implementação foi feita utilizando o vertex shader e o fragment shader do GLSL. Os objetos randerizados aparecem um de cada vez na tela, sendo a troca de objetos controlada pelo teclado. O mesmo vale para os shading models. Usou-se, ao todo, 3 vertex shaders e 3 fragment shaders, sendo um par de cada reservado pra um tipo de shading.</p>

<p><b>Obs:</b> Os objetos estão girando, mas a luz acompanha. Isso foi deixado de propósito para ver como é cada parte do objeto iluminado. Para fazer a luz não acompanhar a rotação, basta utilizar a matriz transposta da inversa da mvMatrix, presente no shader e aplicar a transformação na normal: normal = vec3(matrizResultante * vec4(normal, 1.0f)). Para a rotação, essa matriz é a própria mvMatrix.</p>

<p><b>Variaveis utilizadas pelos shaders</b></p>
<p>As variáveis definidas para poder realizar os algoritmos foram:</p>
<p>-Um vec3 para representar a intensidade de cada cor (rgb), o vetor lightColor.</p>
<p>-Um vec3 para representar a cor do objeto.</p>
<p>-Um vec3 injetado no shader para representar a posição dos vertices do objeto.</p>
<p>-Um vec3 injetado no shader para representar a posiçao da luz na tela do objeto.</p>
<p>-Um vec3 injetado no shader para representar a posiçao do ponto de visão do objeto.</p>
<p>-Um float para representar o coeficiente para ambient reflectance</p>
<p>-Um float para representar o coeficiente para diffuse reflectance</p>
<p>-Um float para representar o coeficiente para specular reflectance</p>

<p><b>Phong Lighting Model:</b></p>
<p>Para implementar o Phong Lighting Model, precisou-se implementar 3 formas de coloração diferentes: ambient, diffuse e specular. O vetor normal foi calculado em diferentes formas pra cada shading model. O vetor posição da luz foi definido pela normalização da posição da luz menos a posicao do vertice analizado. O vetor posição da visão foi definido pela normalização da posição da visão menos a posicao do vertice analizado. O vetor de reflexão foi calculado pela formula 2 * max(N.L, 0)*N - L.</p>
<p>A cor ambiente foi calculada por: intensidade * coeficiente ambient reflectance * cor do objeto.</p>
<p>A cor diffuse foi calculada por: intensidade * coeficiente diffuse * max(N.L, 0)</p>
<p>A cor specular foi calculada por: intensidade * coeficiente specular * pow(max(V.R, 0), 100)</p>
<p>A cor resultante foi calculada pela soma das cores ambiente, diffuse e specular.
<p><b>Obs:</b> A função max foi acrescentada ao produto escalar para garantir que não dê um número negativo</p>

<p><b>Gouraud Shading:</b></p>
<p>Para utilziar o Gouraud Shading, realizou-se os cálculos no vertex shader das cores do modelo de phong. Para realizar esses cálculos, a normal foi definida pela normalização do vetor posiçao do vértice menos a origem (que no caso é (0,0,0)). Por realizar os calculos por vértices, gasta menos processamento que o Phong shading.</p>

<p><b>Phong Shading:</b></p>
<p>Para utilziar o Phong Shading, realizou-se os cálculos no fragment shader das cores do modelo de phong. Para realizar esses cálculos, a normal foi definida pela normalização do vetor posiçao do pixel (que é o vetor posição dos vértices após serem interpolados do vertex shader para o fragment shader) menos a origem (que no caso é (0,0,0)). Por realizar os calculos por fragmento, gasta mais processamento que o Gouraud shading.</p></p>

<p><b>Flat Shading:</b></p>
<p>Para utilziar o Flat Shading, realizou-se os cálculos no fragment shader das cores do modelo de phong. Para realizar esses cálculos, a normal foi definida após o cálculo dos vetores tangentes à posição do pixel, feitas por meio dos métodos dFdx e dFdy do GLSL. Após isso, bastou fazer normalização do produto vetorial entra a tangente de y e a tangente de x. Os cálculos feitos assim acabam gastando mais processamento do que os outros shadings, uma alternativa para realizar o flat shading seria por meio da utilização do geometry shader.

<p><b>Configuração default</b></p>
<p>objeto: sphere;</p>
<p>iluminação: gouraud;</p>

<p><b>Comandos</b></p>
<p><b>s</b>: mudar objeto randerizado para sphere;</p>
<p><b>t</b>: mudar objeto randerizado para teapot;</p>
<p><b>c</b>: mudar objeto randerizado para cylinder.</p>
<p><b>1</b>: mudar iluminação para gouraud shading;</p>
<p><b>2</b>: mudar iluminação para phong shading;</p>
<p><b>3</b>: mudar iluminação para flat shading;</p>

In [1]:
import sys
from OpenGL.GLUT import *
from OpenGL.GL import *
from OpenGL.GLU import *
import OpenGL.GL.shaders

import numpy as np #matrix manipulation
import time #proccess sleep time manipulation
import math # for sin and cos functions

Gouraud Shading shaders

In [2]:
vertex_shader_gouraud = """
\n #version 330 \n
in vec3 position;
out vec4 fragColor;
uniform vec3 lightPosition;
uniform vec3 viewPosition;
uniform mat4 mvMatrix;
void main(){
    //Aux
    vec3 normal = normalize(position);
    vec3 lightDirection = normalize(lightPosition - position);
    //Ambient
    float objReflectance = 0.2f;
    vec3 lightColor = vec3(1.0f, 1.0f, 1.0f);
    vec3 objColor = vec3(1.0f, 0.0f, 0.0f);
    vec3 ambientColor =  objReflectance * lightColor * objColor;
    //Diffuse
    float diffReflectance = 0.4f;
    vec3 diffuseColor = diffReflectance * max(dot(normal, lightDirection), 0.0f) * lightColor * objColor;
    //Specular
    vec3 reflectDirection = 2 * max(dot(normal, lightDirection), 0.0f) * normal - lightDirection;
    vec3 viewDirection = normalize(viewPosition - position);
    float specReflectance = 0.5f;
    vec3 specularColor = specReflectance * pow(max(dot(viewDirection, reflectDirection), 0.0), 100) * lightColor * objColor;
    vec3 resultColor = ambientColor + diffuseColor + specularColor;
    gl_Position = mvMatrix * vec4(position, 1.0f);
    fragColor = vec4(resultColor, 1.0f);
}

"""

fragment_shader_gouraud = """
\n #version 330 \n
out vec4 gl_FragColor;
in vec4 fragColor;
void main(){
    gl_FragColor = fragColor;
}
"""

Phong Shading shaders

In [3]:
vertex_shader_phong = """
\n #version 330 \n
in vec3 position;
out vec3 fragPosition;
out vec3 fragNormal;
uniform mat4 mvMatrix;
void main(){
    gl_Position = mvMatrix * vec4(position, 1.0f);
    fragPosition = position;
    fragNormal = normalize(position);
}

"""

fragment_shader_phong = """
\n #version 330 \n
uniform vec3 lightPosition;
uniform vec3 viewPosition;
out vec4 gl_FragColor;
in vec3 fragPosition;
in vec3 fragNormal;
void main(){
    //Aux
    vec3 lightDirection = normalize(lightPosition - fragPosition);
    //Ambient
    float objReflectance = 0.2f;
    vec3 lightColor = vec3(1.0f, 1.0f, 1.0f);
    vec3 objColor = vec3(1.0f, 0.0f, 0.0f);
    vec3 ambientColor =  objReflectance * lightColor * objColor;
    //Diffuse
    float diffReflectance = 0.4f;
    vec3 diffuseColor = diffReflectance * max(dot(fragNormal, lightDirection), 0.0f) * lightColor * objColor;
    //Specular
    vec3 reflectDirection = 2 * max(dot(fragNormal, lightDirection), 0.0f) * fragNormal - lightDirection;
    vec3 viewDirection = normalize(viewPosition - fragPosition);
    float specReflectance = 0.5f;
    vec3 specularColor = specReflectance * pow(max(dot(viewDirection, reflectDirection), 0.0), 100) * lightColor * objColor;
    vec3 resultColor = ambientColor + diffuseColor + specularColor;
    gl_FragColor = vec4(resultColor, 1.0f);;
}
"""

Flat Shading shaders

In [4]:
vertex_shader_flat = """
\n #version 330 \n
in vec3 position;
out vec3 fragPosition;
uniform mat4 mvMatrix;
void main(){
    gl_Position = mvMatrix * vec4(position, 1.0f);
    fragPosition = position;
}

"""

fragment_shader_flat = """
\n #version 330 \n
uniform vec3 lightPosition;
uniform vec3 viewPosition;
out vec4 gl_FragColor;
in vec3 fragPosition;
void main(){
    //Aux
    vec3 tx = dFdx(fragPosition);
    vec3 ty = dFdy(fragPosition);
    vec3 fragNormal = normalize(cross(ty, tx));
    vec3 lightDirection = normalize(lightPosition - fragPosition);
    //Ambient
    float objReflectance = 0.2f;
    vec3 lightColor = vec3(1.0f, 1.0f, 1.0f);
    vec3 objColor = vec3(1.0f, 0.0f, 0.0f);
    vec3 ambientColor =  objReflectance * lightColor * objColor;
    //Diffuse
    float diffReflectance = 0.4f;
    vec3 diffuseColor = diffReflectance * max(dot(fragNormal, lightDirection), 0.0f) * lightColor * objColor;
    //Specular
    vec3 reflectDirection = 2 * max(dot(fragNormal, lightDirection), 0.0f) * fragNormal - lightDirection;
    vec3 viewDirection = normalize(viewPosition - fragPosition);
    float specReflectance = 0.5f;
    vec3 specularColor = specReflectance * pow(max(dot(viewDirection, reflectDirection), 0.0), 100) * lightColor * objColor;
    vec3 resultColor = ambientColor + diffuseColor + specularColor;
    gl_FragColor = vec4(resultColor, 1.0f);;
}
"""

Funções para o calculo das matrizes de tranformação.

In [ ]:
def getTranslation3DMatrix(tx, ty, tz):
    return [
        [1, 0, 0, tx],
        [0, 1, 0, ty],
        [0, 0, 1, tz],
        [0, 0, 0,  1]
    ]
def getRotataionX3DMatrix(angle):
    seno = math.sin(math.radians(angle)) 
    cosseno = math.cos(math.radians(angle))
    return [
        [1, 0, 0, 0],
        [0, cosseno, -seno, 0],
        [0, seno, cosseno, 0],
        [0, 0, 0, 1]
    ]
def getRotataionY3DMatrix(angle):
    seno = math.sin(math.radians(angle)) 
    cosseno = math.cos(math.radians(angle))
    return [
        [cosseno, 0, seno, 0],
        [0, 1, 0, 0],
        [-seno, 0,cosseno, 0],
        [0, 0, 0, 1]
    ]
def getRotataionZ3DMatrix(angle):
    seno = math.sin(math.radians(angle)) 
    cosseno = math.cos(math.radians(angle))
    return [
        [cosseno, -seno, 0, 0],
        [seno, cosseno, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ]
def getScale3DMatrix(sx, sy, sz):
    return [
        [sx, 0, 0, 0],
        [0, sy, 0, 0],
        [0, 0, sz, 0],
        [0, 0, 0,  1]
    ]
def getIdentity3DMatrix():
    return [
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ]

Código de execução

In [ ]:
def drawPolygon3v(vertexArray):
    glBegin(GL_TRIANGLES)
    length = len(vertexArray)
    for i in range(length):
        glVertex3fv(vertexArray[i])
    glEnd()
    
def draw(): #draws graphic objects on screen
    color = np.array([0.0, 1.0, 0.0]) #green
    # clear all pixels 
    glClear (GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    lightPositionHandle = glGetUniformLocation(currentProgram, "lightPosition")
    glUniform3fv(lightPositionHandle, 1, lightPosition)
    viewPositionHandle = glGetUniformLocation(currentProgram, "viewPosition")
    glUniform3fv(viewPositionHandle, 1, viewPosition)
    mvMatrixHandle = glGetUniformLocation(currentProgram, "mvMatrix")
    glUniformMatrix4fv(mvMatrixHandle, 1, GL_FALSE, mvMatrix)
    drawObjFunc();
    glutSwapBuffers()
    
def drawSphere():
    gluSphere(sphereQuad, 0.5,50,50)
    
def drawTeapot():
    glutSolidTeapot(0.5)
    
def drawCylinder():
    gluCylinder(cylinderQuad,0.3,0.3, 0.5, 20, 5);
    
def update(): #updates graphic
    global shadingTypeChanged
    global objTypeChanged
    global currentProgram
    global drawObjFunc
    global mvMatrix
    if shadingType == 0 and shadingTypeChanged: #gouraud
        glUseProgram(program_gouraud)
        currentProgram = program_gouraud
        shadingTypeChanged = False
    elif shadingType == 1 and shadingTypeChanged: #phong
        glUseProgram(program_phong)
        currentProgram = program_phong
        shadingTypeChanged = False
    elif shadingType == 2 and shadingTypeChanged: #fat
        glUseProgram(program_flat)
        currentProgram = program_flat
        shadingTypeChanged = False
    
    if objType == 0 and objTypeChanged:
        drawObjFunc = drawSphere
        objTypeChanged = False
    elif objType == 1 and objTypeChanged:
        drawObjFunc = drawTeapot
        objTypeChanged = False
    elif objType == 2 and objTypeChanged:
        drawObjFunc = drawCylinder
        objTypeChanged = False
    mvMatrix = mvMatrix @ np.array(getRotataionY3DMatrix(angle))
    
def mainloop(): #glut main loop
    sleepTime = 1.0 / 50
    time.sleep(sleepTime)
    update()
    draw()

def OnKeyboardDownEvent(key, x, y): #updates graphic object states based on keyboard
    global shadingType
    global shadingTypeChanged
    global objType
    global objTypeChanged
    if ord(key) == 49: #1
        if shadingType != 0:
            shadingType = 0
            shadingTypeChanged = True
    elif ord(key) == 50: #2
        if shadingType != 1:
            shadingType = 1
            shadingTypeChanged = True
    elif ord(key) == 51: #3
        if shadingType != 2:
            shadingType = 2
            shadingTypeChanged = True

    elif ord(key) == 83 or ord(key) == 115: #sphere
        if objType != 0:
            objType = 0
            objTypeChanged = True
    elif ord(key) == 84 or ord(key) == 116: #teapot
        if objType != 1:
            objType = 1
            objTypeChanged = True

    elif ord(key) == 67 or ord(key) == 99: #cylinder
        if objType != 2:
            objType = 2
            objTypeChanged = True

def onClose():
    gluDeleteQuadric(sphereQuad)
    gluDeleteQuadric(cylinderQuad)
    
def init():
    global program_gouraud
    global program_phong
    global program_flat
    global currentProgram
    global sphereQuad
    global cylinderQuad
    glEnable(GL_DEPTH_TEST)
    # select clearing color         
    glClearColor (0.0, 0.0, 0.0, 0.0) #black
    # initialize viewing values  
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    a
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    program_gouraud = OpenGL.GL.shaders.compileProgram(OpenGL.GL.shaders.compileShader(vertex_shader_gouraud, GL_VERTEX_SHADER), OpenGL.GL.shaders.compileShader(fragment_shader_gouraud, GL_FRAGMENT_SHADER))
    program_phong = OpenGL.GL.shaders.compileProgram(OpenGL.GL.shaders.compileShader(vertex_shader_phong, GL_VERTEX_SHADER), OpenGL.GL.shaders.compileShader(fragment_shader_phong, GL_FRAGMENT_SHADER))
    program_flat= OpenGL.GL.shaders.compileProgram(OpenGL.GL.shaders.compileShader(vertex_shader_flat, GL_VERTEX_SHADER), OpenGL.GL.shaders.compileShader(fragment_shader_flat, GL_FRAGMENT_SHADER))
    glUseProgram(program_gouraud)
    currentProgram = program_gouraud
    sphereQuad = gluNewQuadric()
    cylinderQuad = gluNewQuadric()
    gluQuadricDrawStyle (cylinderQuad, GLU_FILL);
    
#screenDimensions
weight = 512 
height = 512
depth = 1000

#state vars
shadingType = 0 # 0 - gouraud, 1 - phong, 2 - flat
shadingTypeChanged = False
objType = 0
objTypeChanged = False
currentProgram = -1
drawObjFunc = drawSphere

#gl vars
program_gouraud = -1
program_phong = -1
program_flat = -1

#glsl vars
lightPosition = [-4.0, 0.0, -8.0]
viewPosition = [-4.0, 0.0, -8.0]
mvMatrix = np.array(getIdentity3DMatrix())

#model view transform vars
angle = 0.5

#glut config
glutInit(sys.argv)
glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB)
glutInitWindowSize(weight, height)
glutInitWindowPosition(100, 100)
glutCreateWindow("tp1")
init()
glutDisplayFunc(draw)
glutIdleFunc(mainloop)
glutCloseFunc(onClose)
glutKeyboardFunc(OnKeyboardDownEvent)
glutMainLoop()